In [18]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np

import seaborn as sns
from sklearn.metrics import pairwise

In [19]:
ds = tfds.load('multi_news', split='train', with_info=True)

#use index to get specific document
index = 20
count = 0
with tf.Graph().as_default():
    numpy_imgs = next(iter(ds))
    # numpy_imgs = tfds.as_numpy(ds)

In [20]:
count = 0
document = []
summary = []
for x in numpy_imgs:
    count += 1
    if count == index:
        # tf.print(x["document"])
        # print("\n")
        # print("\n")
        # print("SUMMARY")
        # tf.print(x["summary"])

        document = x["document"]
        summary = x["summary"]
        break

In [21]:
document = bytes(document.numpy())
document = [document.decode("utf-8")]

summary = bytes(summary.numpy())
summary = [summary.decode("utf-8")]
print(summary)

['– Umberto Eco started working on a novel that set the world\'s imagination on fire "prodded by a seminal idea: I felt like poisoning a monk." The Italian author and academic who intrigued, puzzled, and delighted readers worldwide with his best-selling historical novel The Name of the Rose died at home in Milan on Friday evening after a battle with cancer, a family member tells the AP. He was 84. Author of a wide range of books, Eco was fascinated with the obscure and the mundane, and his books were both engaging narratives and philosophical and intellectual exercises. The Name of the Rose transformed him from an academic to international celebrity, especially after the medieval thriller set in a monastery was made into a film starring Sean Connery in 1986. His second novel, 1988\'s Foucault\'s Pendulum, a byzantine tale of plotting publishers and secret sects, was successful, too—though it was so complicated that an annotated guide accompanied it to help the reader follow the plot. E

In [ ]:
# stuff to do
# get vocabulary
# display output in a visual way
#lean word2vec

preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert = hub.load('https://tfhub.dev/google/experts/bert/wiki_books/2')

sentences = [
  "Here We Go Then, You And I is a 1999 album by Norwegian pop artist Morten Abel. It was Abel's second CD as a solo artist.",
  "The album went straight to number one on the Norwegian album chart, and sold to double platinum.",
  "Among the singles released from the album were the songs \"Be My Lover\" and \"Hard To Stay Awake\".",
  "Riccardo Zegna is an Italian jazz musician.",
  "Rajko Maksimović is a composer, writer, and music pedagogue.",
  "One of the most significant Serbian composers of our time, Maksimović has been and remains active in creating works for different ensembles.",
  "Ceylon spinach is a common name for several plants and may refer to: Basella alba Talinum fruticosum",
  "A solar eclipse occurs when the Moon passes between Earth and the Sun, thereby totally or partly obscuring the image of the Sun for a viewer on Earth.",
  "A partial solar eclipse occurs in the polar regions of the Earth when the center of the Moon's shadow misses the Earth.",
]

wordArray = []

for i in sentences:
  words = i.split()
  for w in words:
    w = w.replace(",", "")
    w = w.replace(".", "")
    wordArray.append(w)

print("word array")
print(wordArray)

bert_inputs = preprocess(sentences)
bert_outputs = bert(bert_inputs)
pooled_output = bert_outputs['pooled_output']
sequence_output = bert_outputs['sequence_output']

print('\nPooled output:')
print(pooled_output)
print('\nSequence output:')
print(sequence_output)


In [ ]:
def plot_similarity(features, labels):
  """Plot a similarity matrix of the embeddings."""
  cos_sim = pairwise.cosine_similarity(features)
  sns.set(font_scale=1.2)
  cbar_kws=dict(use_gridspec=False, location="left")
  g = sns.heatmap(
      cos_sim, xticklabels=labels, yticklabels=labels,
      vmin=0, vmax=1, cmap="Blues", cbar_kws=cbar_kws)
  g.tick_params(labelright=True, labelleft=False)
  g.set_yticklabels(labels, rotation=0)
  g.set_title("Semantic Textual Similarity")

plot_similarity(bert_outputs["pooled_output"], wordArray)